In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import shutil
import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

In [ ]:
labelmap = pd.read_csv('../input/labels.csv')
labelindex = labelmap.breed.unique()

#Make all the necessary folders for keras generator 
#to train the model

os.mkdir("data")
os.mkdir("data/train")
os.mkdir("data/valid")
for label in labelindex:
    os.mkdir("data/train/" + label)
    os.mkdir("data/valid/" +  label)

    

In [ ]:
def findclass(filename):
    #get the class name from csvdata
    classname = labelmap.loc[labelmap['id'] == filename]
    classname = classname['breed'].values.tolist()
#     print(classname,filename)
#     #check the index of the list
#     classindex = np.where(labelindex == classname[0])
#     print(classindex)
    return classname[0]
    
def createvalidationdata(labelindex,labelmap,files):
    train_images = []
    valid_images= []
    total = 0
    for label in labelindex:
        labelsamples = labelmap.loc[labelmap['breed'] == label]
        labelsamples = labelsamples.id.values.tolist()
        total = total + len(labelsamples)
        trainlabelcut = int((np.floor(len(labelsamples) * 0.9)))
        trainlabelsamples = labelsamples[0:trainlabelcut]
        validlabelsamples = labelsamples[trainlabelcut:len(labelsamples)]
        train_images = train_images + trainlabelsamples
        valid_images = valid_images + validlabelsamples
        

    print(total)
    return train_images,valid_images   

root, subdir,files = next(os.walk("../input/train/"))
np.random.shuffle(files)
train_images,valid_images = createvalidationdata(labelindex,labelmap,files)

print(len(train_images),len(valid_images))
#copy train images to their respective class folders in train dir
for image in train_images:
    label_class = findclass(image)
    shutil.copy2("../input/train/" + image + ".jpg","../working/data/train/" + label_class)

#copy valid images to their respective class folders in validation dir
for image in valid_images:
    label_class = findclass(image)
    shutil.copy2("../input/train/" + image + ".jpg","../working/data/valid/" + label_class)

print("Files Copied")

In [ ]:

img_width, img_height = 256, 256
train_data_dir = "../working/data/train"
validation_data_dir = "../working/data/valid"
nb_train_samples = len(train_images)
nb_validation_samples = len(valid_images) 
batch_size = 32
epochs = 50
num_classes = 120

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
model.summary()


In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:10]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)
model_final.summary()
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [ ]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg120_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])